In [2]:
!pip -qqq install yt-dlp

In [3]:
from yt_dlp import YoutubeDL
import os
import re

In [ ]:
#!pip install moviepy
#!pip install pytube

In [8]:
from pytube import YouTube
from moviepy import VideoFileClip

In [9]:
def download_youtube_video(url, download_path='downloads_vdo/'):
    # Create directory if it doesn't exist
    if not os.path.exists(download_path):
        os.makedirs(download_path)

    # Download the YouTube video
    yt = YouTube(url)
    video = yt.streams.filter(only_video=False, file_extension='mp4').first()
    video_path = video.download(output_path=download_path)

    print(f"Video downloaded: {video_path}")
    return video_path

In [10]:
video_urls = ["https://www.youtube.com/watch?v=qP1JKWBBy80",
                "https://www.youtube.com/watch?v=_p58cZIHDG4"]

In [ ]:
#for video in video_urls:
    #download_youtube_video(video)

In [ ]:
video_path = "/home/pms/llm_project/mm_rag_esg_financial_project/downloads_vdo/Invest_with_Conscience_ ESG_Investing.mp4"
audio_path = os.path.splitext(video_path)[0] + '.mp3'

video_clip = VideoFileClip(video_path)
video_clip.audio.write_audiofile(audio_path,codec='mp3')
video_clip.close()


In [ ]:
# class YouTubeAudioDownloader:
#     def __init__(self, output_folder):
#         self.output_folder = os.path.abspath(output_folder)
#         self.audio_files_dict = {}

#     def get_safe_filename(self, filename):
#         safe_filename = re.sub(r'[^\w\-.]', '_', filename)
#         safe_filename = re.sub(r'_+', '_', safe_filename)
#         safe_filename = safe_filename[:50].strip('_')
#         return safe_filename

#     def download_audio(self, video_url):
#         try:
#             ydl_opts = {
#                 'format': 'bestaudio/best',
#                 'postprocessors': [{
#                     'key': 'FFmpegExtractAudio',
#                     'preferredcodec': 'mp3',
#                     'preferredquality': '192',
#                 }],
#                 'outtmpl': os.path.join(self.output_folder, '%(title)s.%(ext)s'),
#             }

#             with YoutubeDL(ydl_opts) as ydl:
#                 info = ydl.extract_info(video_url, download=True)
#                 filename = ydl.prepare_filename(info)
#                 base, ext = os.path.splitext(filename)
#                 new_file = base + '.mp3'

#             print(f"Audio file downloaded: {new_file}")
#             self.audio_files_dict[video_url] = new_file
#             return new_file
#         except Exception as e:
#             print(f"Error downloading audio from {video_url}: {str(e)}")
#             return None

#     def download_multiple_audios(self, video_urls):
#         for url in video_urls:
#             print(f"Processing video: {url}")
#             audio_file = self.download_audio(url)
#             if audio_file is None:
#                 print(f"Failed to download audio from video: {url}")
#         return self.audio_files_dict

In [ ]:
# downloader = YouTubeAudioDownloader(output_folder="./data")
# video_urls = ["https://www.youtube.com/watch?v=qP1JKWBBy80",
#                 "https://www.youtube.com/watch?v=_p58cZIHDG4"]

# # Download audios from multiple videos
# audio_files = downloader.download_multiple_audios(video_urls)

# print("Downloaded audio files:")
# for audio_file in audio_files:
#     print(audio_file)

### Transcriptions Generation with Whisper

In [ ]:
!pip -qqq install openai-whisper

In [2]:
import whisper
import torch

In [9]:
class AudioTranscriber:
    def __init__(self, input_folder):
        self.input_folder = os.path.abspath(os.path.join(os.getcwd(), input_folder))
        self.whisper_model = None
        self.transcriptions_dict = {}

    def transcribe_audio(self, audio_file):
        try:
            if not os.path.exists(audio_file):
                print(f"Audio file not found: {audio_file}")
                return None

            file_size = os.path.getsize(audio_file)
            if file_size == 0:
                print(f"Audio file is empty: {audio_file}")
                return None

            transcription = self.whisper_model.transcribe(audio_file)
            return transcription["text"]
        except Exception as e:
            print(f"Error in transcribe_audio: {str(e)}")
            return None

    def transcribe_all_audios(self, audio_files_dict):
        # for url, audio_path in audio_files_dict.items():
        #     if not audio_path.endswith('.mp3'):
        #         print(f"Skipping non-mp3 file: {audio_path}")
        #         continue

        transcription = self.transcribe_audio(audio_path)

        if transcription is not None:
            # Add to transcriptions dictionary
            self.transcriptions_dict = {
                #'url': url,
                'audio_path': audio_path,
                'transcription': transcription
            }
        else:
            print(f"Failed to transcribe audio: {audio_path}")

        return self.transcriptions_dict

In [4]:
device = "cude" if torch.cuda.is_available() else "cpu"
whisper_model = whisper.load_model("medium",device=device)

/home/pms/llm_project/mm_rag_esg_financial_project/esg/lib/python3.10/site-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch

In [ ]:

mp3files = []



['/home/pms/llm_project/mm_rag_esg_financial_project/data/Invest_with_Conscience_ ESG_Investing.mp3', '/home/pms/llm_project/mm_rag_esg_financial_project/data/ESG_investing_complete.mp3']


In [5]:
import glob
# Initialize the AudioTranscriber
#transcriber = AudioTranscriber(input_folder=r"./data")

# Initialize your Whisper model
#transcriber.whisper_model = whisper_model

# Transcribe all audios in the input folder
#for file in glob.glob(f'{os.getcwd()}/data/*.mp3'):
transcriptions_dict = whisper_model.transcribe("/home/pms/llm_project/mm_rag_esg_financial_project/data/ESG_investing_complete.mp3")

#print(transcriptions_dict)
#transcriptions_dict

# for _, data in transcriptions_dict.items():
#     #print(f"URL: {url}")
#     print(f"Audio file: {data['audio_path']}")
#     print(f"Transcription: {data['transcription'][:100]}...")  # Print first 100 characters
#     print("---")

/home/pms/llm_project/mm_rag_esg_financial_project/esg/lib/python3.10/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


In [6]:
transcriptions_dict['text']

" ESG, real or marketing? It's a complete fraud. Complete fraud. It's so ridiculous. Governance has been addressed. That's useful. But this idea that you're going to get a stamp that says, oh, listen, my supplier, I've offset their carbon credits and now I understand my... It's a joke. It's jargon. And I think what people are doing right now is using it as a way to, for example, if you can paint yourself as ESG, in Europe you can essentially borrow money from the ECB at negative rates. I can do you a massage now. It's a carry-trade. He doesn't want you touching him. Coronavirus. Go Chamath. Go Chamath. Hold on. But I personally believe in climate change. I know. I think we need to do something. And so the problem with ESG is it's going to take years for this... But when you hear JP Morgan yesterday say they're not going to finance fossil fuels or you hear Ed Bastien at Delta say he's going to spend $100 million, real money by the way, effectively buying carbon offsets and investing in 

### PDF Dpcument Parsing

In [7]:
from unstructured.partition.pdf import partition_pdf

In [8]:
raw_pdf_elements=partition_pdf(
    filename="/home/pms/llm_project/mm_rag_esg_financial_project/pdf/Bidding_Document_UWS_AGRC.pdf",
    strategy="hi_res",
    extract_images_in_pdf=True,
    extract_image_block_types=["Image", "Table"],
    extract_image_block_to_payload=False,
    extract_image_block_output_dir="extracted_data",
)

In [ ]:
raw_pdf_elements

In [10]:
Header=[]
Footer=[]
Title=[]
NarrativeText=[]
Text=[]
ListItem=[]
img=[]
tab=[]
for element in raw_pdf_elements:
  if "unstructured.documents.elements.Header" in str(type(element)):
            Header.append(str(element))
  elif "unstructured.documents.elements.Footer" in str(type(element)):
            Footer.append(str(element))
  elif "unstructured.documents.elements.Title" in str(type(element)):
            Title.append(str(element))
  elif "unstructured.documents.elements.NarrativeText" in str(type(element)):
            NarrativeText.append(str(element))
  elif "unstructured.documents.elements.Text" in str(type(element)):
            Text.append(str(element))
  elif "unstructured.documents.elements.ListItem" in str(type(element)):
            ListItem.append(str(element))
  elif "unstructured.documents.elements.Image" in str(type(element)):
            img.append(str(element))
  elif "unstructured.documents.elements.Table" in str(type(element)):
            tab.append(str(element))

In [12]:
Header

['Section 1: Instructions to Bidders',
 '1-1',
 '1-2',
 'Section 1: Instructions to Bidders',
 'Section 1: Instructions to Bidders',
 '1-3',
 '1-4',
 'Section 1: Instructions to Bidders',
 'Section 1: Instructions to Bidders',
 '1-5',
 '1-6',
 'Section 1: Instructions to Bidders',
 'Section 1: Instructions to Bidders',
 '1-7',
 'Section 1: Instructions to Bidders',
 'Section 1: Instructions to Bidders',
 '1-9',
 '1-10',
 'Section 1: Instructions to Bidders',
 'Section 1: Instructions to Bidders',
 '1-11',
 '1-12',
 'Section 1: Instructions to Bidders',
 'Section 1: Instructions to Bidders',
 '1-13',
 '1-14',
 'Section 1: Instructions to Bidders',
 'Section 1: Instructions to Bidders',
 '1-15',
 '1-16',
 'Section 1: Instructions to Bidders',
 'Section 1: Instructions to Bidders',
 '1-17',
 '1-18',
 'Section 1: Instructions to Bidders',
 'Section 1: Instructions to Bidders',
 '1-19',
 '1-20',
 'Section 1: Instructions to Bidders',
 'Section 1: Instructions to Bidders',
 '1-21',
 '1-22',


In [11]:
tab

['Issued on : 15.08.2024 Invitation for Bids No. : OCB No. : GD-2 (SI-02) Purchaser : Titas Gas Transmission and Distribution PLC Country : Bangladesh',
 'Instructions to Bidders ............................................................................................... 1-1 Bid Data Sheet .......................................................................................................... 2-1 Evaluation and Qualification Criteria ......................................................................... 3-1 Bidding Forms ........................................................................................................... 4-1 Eligible Countries ....................................................................................................... 5-1',
 'Section 6. Schedule of REQUiIFEMENtS oo... eee eee ete este ete tet eteceteetetteeeeteeteteetesietietesieneeeeee 6-1',
 'General Conditions of Contract ......................................................................